In [1]:
%load_ext autoreload
import numpy as np
import os
import plotly.graph_objects as go
from kernels import IonRate
from field_functions import LaserField

class AU:
    meter = 5.2917720859e-11 # atomic unit of length in meters
    nm = 5.2917721e-2 # atomic unit of length in nanometres
    second = 2.418884328e-17 # atomic unit of time in seconds
    fs = 2.418884328e-2 # atomic unit of time in femtoseconds
    Joule = 4.359743935e-18 # atomic unit of energy in Joules
    eV = 27.21138383 # atomic unit of energy in electronvolts
    Volts_per_meter = 5.142206313e+11 # atomic unit of electric field in V/m
    Volts_per_Angstrom = 51.42206313 # atomic unit of electric field in V/Angström
    speed_of_light = 137.035999 # vacuum speed of light in atomic units
    Coulomb = 1.60217646e-19 # atomic unit of electric charge in Coulombs
    PW_per_cm2_au = 0.02849451308 # PW/cm^2 in atomic units
AtomicUnits=AU



params = {'E_g': 0.5, 'αPol': 4.51, "div_p":2**-4*2, "div_theta":1, 'lam0': 850, 'intensity': 1e14, 'cep': 0}

In [2]:
%autoreload
lam0 = params['lam0']
intensity = params['intensity']
cep = params['cep']
laser_pulses = LaserField(cache_results=True)
laser_pulses.add_pulse(lam0, intensity, cep, lam0/ AtomicUnits.nm / AtomicUnits.speed_of_light)
t_min, t_max = laser_pulses.get_time_interval()
time_recon= np.arange(int(t_min), int(t_max)+1, 1.)

ion_na_rate_SFA = IonRate(time_recon, laser_pulses, params, dT=0.5, kernel_type='exact_SFA')
laser_pulses.reset()

In [3]:
%autoreload
lam0 = params['lam0']
intensity = params['intensity']
cep = params['cep']
laser_pulses = LaserField(cache_results=True)
laser_pulses.add_pulse(lam0, intensity, cep, lam0/ AtomicUnits.nm / AtomicUnits.speed_of_light)
t_min, t_max = laser_pulses.get_time_interval()
time_recon= np.arange(int(t_min), int(t_max)+1, 1.)

ion_na_rate_SFA_excited_tRecX = IonRate(time_recon, laser_pulses, params, dT=0.5, kernel_type='exact_SFA', excitedStates=3, coeffType="trecx") #it uses the wrong coefficients for the calculation, it uses the 2s state for excitedSTates=2 and 2s is not even there
laser_pulses.reset()

In [4]:
%autoreload
lam0 = params['lam0']
intensity = params['intensity']
cep = params['cep']
laser_pulses = LaserField(cache_results=True)
laser_pulses.add_pulse(lam0, intensity, cep, lam0/ AtomicUnits.nm / AtomicUnits.speed_of_light)
t_min, t_max = laser_pulses.get_time_interval()
time_recon= np.arange(int(t_min), int(t_max)+1, 1.)
# to save computation time we can neglect cross terms!! just look at formula for <p|d|psi> there is i^l and because of the transition rules l has to be +-1 so if we sum over all states and one is complex conjugatet and we sum it up they will cancel out !!! but be carefull we only can use states that are allowed theoretically


ion_na_rate_SFA_excited_numerical = IonRate(time_recon, laser_pulses, params, dT=0.5, kernel_type='exact_SFA', excitedStates=3, coeffType="numerical", gauge="length") #it uses the wrong coefficients for the calculation, it uses the 2s state for excitedSTates=2 and 2s is not even there
laser_pulses.reset()

[autoreload of coefficientsNumerical failed: Traceback (most recent call last):
  File "/home/user/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/user/.local/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/user/BachelorThesis/Bachelor-thesis/ionModel/python/coefficientsNumerical.py", line 16, in <module>
    class HydrogenSolver:
  File "/home/user/BachelorThesis/Bachelor-thesis/ionModel/python/coefficientsNumerical.py", line 63, in HydrogenSolver
    @profile
NameError: name 'profile' is no

Basis states (6): [(1, 0, 0), (2, 0, 0), (2, 1, 0), (3, 0, 0), (3, 1, 0), (3, 2, 0)]


In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_recon, y=np.real(ion_na_rate_SFA_excited_tRecX), mode='lines', name='SFA_excited_tRecX'))
fig.add_trace(go.Scatter(x=time_recon, y=np.real(ion_na_rate_SFA_excited_numerical), mode='lines', name='SFA_excited_tRecX_1s'))
fig.add_trace(go.Scatter(x=time_recon, y=ion_na_rate_SFA, mode='lines', name='SFA'))

fig.update_layout(
    title=f'Ionization Rate for {lam0} nm, {intensity:.2e} W/cm²',
    xaxis_title='Time (a.u.)',
    yaxis_title='Ionization Rate (a.u.)',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    xaxis_range=[-100, 100],
    width=800,
    height=600
)
# truncate y-axis
fig.show()

filename = f"plots/ionRate_{params['lam0']}_c0-is-1.html"
if not os.path.exists(filename):
    fig.write_html(filename)
    print(f"Plot saved as {filename}")
else:
    print(f"File {filename} already exists - skipping save")

Plot saved as plots/ionRate_850_c0-is-1.html
